# Analysis of BERT's knowledge of the Italian Subjunctive

### Imports

In [272]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModelForCausalLM
import re
from datasets import load_dataset

### Functions for corpus creation and extraction of sentences with specific words

In [285]:
# Returns a list of strings created from the opus100 dataset
def create_corpus():
    corpus = []
    for entry in dataset['train']:
        corpus.append(entry['translation']['it'])


    sentence_pattern = re.compile(r'[^.!?]*[.!?]')
    sentences = []
    for entry in corpus:
        sents = sentence_pattern.findall(entry)
        for sent in sents:
            sentences.append(sent.strip())
    return sentences


# Given a list of strings, returns a list of all the strings that contain the word
def find_instances(corpus, word):
    instances = []
    for sentence in corpus:
        pattern = re.compile(fr'\b{re.escape(word)}\b') #, flags=re.IGNORECASE
        if pattern.findall(sentence):
            instances.append(sentence.strip())
    for sentence in instances:
        if word.upper() in sentence:
            instances.remove(sentence)
    return instances


# Returns tokenized sentence, replacing specified token with mask token id
def tokenize_and_mask(sentence, word):
    
    mapping = tokenizer(sentence, return_tensors='pt')
    tokens = mapping['input_ids'][0]
    
    mask_id = bert.tokenizer.mask_token_id
    word_id_lower = tokenizer.convert_tokens_to_ids(word.lower())
    word_id_cap = tokenizer.convert_tokens_to_ids(word.capitalize())
    
    for index, value in enumerate(tokens):
        if value.item() == word_id_lower or value.item() == word_id_cap:
            tokens[index] = mask_id
            break
    mapping['input_ids'][0] = tokens
    return mapping



In [180]:
dataset = load_dataset("opus100", "en-it")
corpus = create_corpus()

In [330]:
se_constructions = ["Se fosse un'infermiera, non avrebbe fatto un intervento chirurgico.",
             "Se fosse più buio, avremmo un motivo per restare in casa.",
             "Se fosse ricco, comprerebbe una casa.",
             "Se non fosse tedesco, le piacerebbe questa insalata.",
             "Se non fosse così timido, parlerebbe con te.",
             "Se non fosse la direttrice, avrebbe più tempo."
]
verb_constructions = ["Non credo che sia tedesco.",
            "Penso che sia già molto vecchio.",
            "Non penso che sia già molto vecchio.",
            "Credo che sia già molto vecchio.",
            "Non credo che sia vero che non possiamo farlo."]
adjective_constructions = ["È possibile che domani sia domenica.",
            "È possibile che lo sia.",
            "È impossibile che sia milionario.",
            "È probabile che sia in ritardo.",
            "È possibile che sia ora di andare.",
            "Non è possibile che domani sia domenica.",
            "Non è possibile che lo sia.",
            "Non è impossibile che sia un millenario.",
            "Non è possibile che sia in ritardo.",
            "Non è possibile che sia ora di andare."]

In [284]:
class Bert():
    
    def __init__(self, name, mode, tokenizer):
        self.name = name
        self.model = model
        self.tokenizer = tokenizer
        self.mask_id = tokenizer.mask_token_id
        self.mask = tokenizer.mask_token
        
    def tokenize(self, sentence):
        tokenized_input = tokenizer(sentence, return_tensors='pt')
        return tokenized_input
    
    def forward(self, tokens):
        output = self.model(**tokens)
        return output
    
    def logits(self, output, mask_location):
        logits = F.softmax(output.logits[:,mask_location,:], dim=1).squeeze()
        return logits
    
    def token_prob(self, logits, token):
        return logits[tokenizer.convert_tokens_to_ids(token)].item()

In [282]:
# model = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-cased")
# tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
# bert = Bert("bert", model, tokenizer)
# mask_id = bert.tokenizer.mask_token_id

model = AutoModelForMaskedLM.from_pretrained("dbmdz/bert-base-italian-cased")
tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-italian-cased')
bert = Bert("bert", model, tokenizer)
mask_id = bert.tokenizer.mask_token_id

Some weights of the model checkpoint at dbmdz/bert-base-italian-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at dbmdz/bert-base-italian-cased and are newly initialized: ['cls.predictions.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [283]:
# Given a sentence, this function masks a specified token and returns the
# probability assigned to a token for that position

def test_sentence(sentence, word_to_mask, word_to_test):
    tokenized = tokenize_and_mask(sentence, word_to_mask)
    try:
        mask_location = list(tokenized['input_ids'][0]).index(mask_id)
    except:
        return None
    return bert.token_prob(bert.logits(bert.forward(tokenized), mask_location), word_to_test)



In [12]:
test_sentence('Non è necessario che [MASK] così.', 'sia', 'è')

0.20595484972000122

In [40]:
test_sentence('Non è necessario che [MASK] così.', 'sia', 'sia')

0.072059266269207

In [23]:
test_sentence('Temo che lui sia malato.', 'sia', 'sia')

0.5012457370758057

In [24]:
test_sentence('Temo che lui sia malato.', 'sia', 'è')

0.4294617176055908

In [27]:
a = bert.forward(bert.tokenize('Non credo che [MASK] veramente tedesco.'))
b = bert.logits(a, 5)
c = bert.token_prob(b, 'sia')


In [28]:
bert.token_prob(b, 'sia')

0.019019730389118195

In [29]:
bert.token_prob(b, 'è')

0.7472403049468994

In [30]:
test_sentence('Non credo che sia veramente tedesco.', 'sia', 'sia')

0.019019730389118195

## Experiment 1: Quantitative Analysis

Comparison of accuracy for word pairs of contrasting grammatical mood

In [197]:
import random

def replace_compare(corpus, n, word_1, word_2):

    corp_1 = find_instances(corpus, word_1)
    corp_1 = random.sample(corp_1, n)
    corp_1_n_correct = 0
    
    odds = 0
    odds_correct = 0
    odds_false = 0
    
    congruent_probs = 0
    incongruent_probs = 0
    
    correct_prob_congruent = 0
    correct_prob_incongruent = 0
    false_prob_congruent = 0
    false_prob_incongruent = 0
    
    samples = []

    for sentence in corp_1:
        correct_prob = test_sentence(sentence, word_1, word_1)
        false_prob = test_sentence(sentence, word_1, word_2)
        if correct_prob == None:
            #print(sentence)
            continue
        if false_prob == None:
            #print(sentence)
            continue

        congruent_probs += correct_prob
        incongruent_probs += false_prob
        
        odd = correct_prob / false_prob
        odds += odd

        if correct_prob >= false_prob:
            corp_1_n_correct += 1
            odds_correct += odd
            correct_prob_congruent += correct_prob
            false_prob_congruent += false_prob
        else:
            samples.append(sentence)
            odds_false += odd
            correct_prob_incongruent += correct_prob
            false_prob_incongruent += false_prob
            
    corp_1_n_false = len(corp_1)-corp_1_n_correct
    if corp_1_n_false == 0:
        corp_1_n_false = 0.0000001
    if corp_1_n_correct == 0:
        corp_1_n_correct = 0.0000001
        
    print(f"### {word_1} was replaced ###")       
    print(f'Accuracy when replacing "{word_1}": {100*corp_1_n_correct/len(corp_1)}%')
    print(f'The average odds is: {odds/len(corp_1)}\n')
    
    print(f'The average odds when predicting correctly is: {odds_correct/corp_1_n_correct}')
    print(f'The average odds when predicting falsely is: {odds_false/corp_1_n_false}\n')
    
    print(f'The average probability for {word_1} was: {congruent_probs/n}')
    print(f'The average probability for {word_2} was: {incongruent_probs/n}\n')
    
    print(f'The average probability for {word_1} when predicting correctly is: {correct_prob_congruent/corp_1_n_correct}')
    print(f'The average probability for {word_2} when predicting correctly is: {false_prob_congruent/corp_1_n_correct}')
    print(f'The average probability for {word_1} when predicting falsely is: {correct_prob_incongruent/corp_1_n_false}')
    print(f'The average probability for {word_2} when predicting falsely is: {false_prob_incongruent/corp_1_n_false}\n\n')

    if len(samples) > 5:
        return samples[:5]
    return samples
    
    

def experiment_1(corpus, word_1, word_2, n):
    samples = []
    samples.append(replace_compare(corpus, n, word_1, word_2))
    samples.append(replace_compare(corpus, n, word_2, word_1))
    print(samples)
    

In [220]:
# replace_compare(corpus, 5, 'sia', 'è')


tokenized = tokenize_and_mask("Claro che sia cosi", 'sia')
try:
    mask_location = list(tokenized['input_ids'][0]).index(mask_id)
except:
    print(3)

#return bert.token_prob(bert.logits(bert.forward(tokenized), mask_location), word_to_test)
#print(test_sentence("Claro sia eso", 'sia', 'è'))
tokenized['input_ids'][0]
tokenizer("Claro che sia cosi")['input_ids']

3


[102, 9354, 30879, 158, 496, 1099, 103]

In [208]:
mask_id

104

In [127]:
experiment_1(corpus, 'sia', 'è', 500)

### sia was replaced ###
Accuracy when replacing "sia": 35.0%
The average odds is: 2065.6235220069034

The average odds when predicting correctly is: 5901.331832324423
The average odds when predicting falsely is: 0.24212414362338394

The average probability for sia was: 0.22937348860322762
The average probability for è was: 0.2905929743952131

The average probability for sia when predicting correctly is: 0.5176749309159331
The average probability for è when predicting correctly is: 0.08616592725202987
The average probability for sia when predicting falsely is: 0.07413425043484774
The average probability for è when predicting falsely is: 0.40066907670308094


### è was replaced ###
Accuracy when replacing "è": 98.2%
The average odds is: 3872.287146190136

The average odds when predicting correctly is: 3943.2568439179477
The average odds when predicting falsely is: 0.49585903949949794

The average probability for è was: 0.6171230605177229
The average probability for sia was: 0.0178996978

In [292]:
experiment_1(corpus, 'sia', 'è', 500)

### sia was replaced ###
Accuracy when replacing "sia": 87.4%
The average odds is: 257532.7700648396

The average odds when predicting correctly is: 294659.8732598451
The average odds when predicting falsely is: 0.3240931340896871

The average probability for sia was: 0.6681543536759255
The average probability for è was: 0.08169309014758384

The average probability for sia when predicting correctly is: 0.7480271711584325
The average probability for è when predicting correctly is: 0.03693637678880792
The average probability for sia when predicting falsely is: 0.11411592129726549
The average probability for è when predicting falsely is: 0.3921483875727438


### è was replaced ###
Accuracy when replacing "è": 99.0%
The average odds is: 10544887.627887057

The average odds when predicting correctly is: 10651401.64209165
The average odds when predicting falsely is: 0.22163216931490987

The average probability for è was: 0.7399559748931497
The average probability for sia was: 0.0111902934045

In [293]:
experiment_1(corpus, 'erano', 'fossero', 500)

### erano was replaced ###
Accuracy when replacing "erano": 98.6%
The average odds is: 773511.5381798766

The average odds when predicting correctly is: 784494.4557406089
The average odds when predicting falsely is: 0.344259742756987

The average probability for erano was: 0.3944257938259613
The average probability for fossero was: 0.005581064003156828

The average probability for erano when predicting correctly is: 0.39947804181705787
The average probability for fossero when predicting correctly is: 0.0030768670461051354
The average probability for erano when predicting falsely is: 0.03860318531016154
The average probability for fossero when predicting falsely is: 0.18194807826408319


### fossero was replaced ###
Accuracy when replacing "fossero": 87.6%
The average odds is: 4237.2226271948675

The average odds when predicting correctly is: 4836.96323074092
The average odds when predicting falsely is: 0.34546020823191453

The average probability for fossero was: 0.4605695652392378
The

In [286]:
experiment_1(corpus, 'hanno', 'abbiano', 500)

### hanno was replaced ###
Accuracy when replacing "hanno": 99.4%
The average odds is: 4370.148054335331

The average odds when predicting correctly is: 4396.525060785973
The average odds when predicting falsely is: 0.3573190121690489

The average probability for hanno was: 0.5989780302036254
The average probability for abbiano was: 0.005269699375920844

The average probability for hanno when predicting correctly is: 0.6020735677570072
The average probability for abbiano when predicting correctly is: 0.004388296684202366
The average probability for hanno when predicting falsely is: 0.0861506421933882
The average probability for abbiano when predicting falsely is: 0.15128874530394873


### abbiano was replaced ###
Accuracy when replacing "abbiano": 75.2%
The average odds is: 65.29118582974237

The average odds when predicting correctly is: 86.74978852614159
The average odds when predicting falsely is: 0.22316475033815114

The average probability for abbiano was: 0.4315007672277218
The a

In [289]:
experiment_1(corpus, 'fanno', 'facciano', 100)

### fanno was replaced ###
Accuracy when replacing "fanno": 98.0%
The average odds is: 617.0538209817636

The average odds when predicting correctly is: 629.6392379114361
The average odds when predicting falsely is: 0.3683914278116088

The average probability for fanno was: 0.35017949109349955
The average probability for facciano was: 0.015046269362894264

The average probability for fanno when predicting correctly is: 0.3570040375477767
The average probability for facciano when predicting correctly is: 0.006170951299311751
The average probability for fanno when predicting falsely is: 0.01577671483391896
The average probability for facciano when predicting falsely is: 0.44993685447843745


### facciano was replaced ###
Accuracy when replacing "facciano": 79.0%
The average odds is: 208.10915503976065

The average odds when predicting correctly is: 263.34629128013944
The average odds when predicting falsely is: 0.31230918309753697

The average probability for facciano was: 0.444152179996

In [291]:
experiment_1(corpus, 'possono', 'possano', 500)

### possono was replaced ###
Accuracy when replacing "possono": 99.6%
The average odds is: 13202.235687668732

The average odds when predicting correctly is: 13255.256354784531
The average odds when predicting falsely is: 0.08957583449661806

The average probability for possono was: 0.613292759815241
The average probability for possano was: 0.010844743241146208

The average probability for possono when predicting correctly is: 0.6154589915019841
The average probability for possano when predicting correctly is: 0.007588126083768636
The average probability for possono when predicting falsely is: 0.07390106981620193
The average probability for possano when predicting falsely is: 0.8217424154281616


### possano was replaced ###
Accuracy when replacing "possano": 80.0%
The average odds is: 146.05458513831636

The average odds when predicting correctly is: 182.50351341547554
The average odds when predicting falsely is: 0.2588720296794394

The average probability for possano was: 0.564527792

In [227]:
experiment_1(corpus, 'sono', 'siano', 500)

### sono was replaced ###
Accuracy when replacing "sono": 99.6%
The average odds is: 96810.41422923023

The average odds when predicting correctly is: 97199.20977429043
The average odds when predicting falsely is: 0.3235092434494197

The average probability for sono was: 0.64510771794626
The average probability for siano was: 0.007705065710451294

The average probability for sono when predicting correctly is: 0.6471200879275129
The average probability for siano when predicting correctly is: 0.005012412849106631
The average probability for sono when predicting falsely is: 0.1440275926142931
The average probability for siano when predicting falsely is: 0.6781756281852722


### siano was replaced ###
Accuracy when replacing "siano": 83.6%
The average odds is: 441.90250495182244

The average odds when predicting correctly is: 528.5432996574172
The average odds when predicting falsely is: 0.24577096476684834

The average probability for siano was: 0.6545038382494749
The average probability 

In [228]:
experiment_1(corpus, 'fossero', 'erano', 500)

### fossero was replaced ###
Accuracy when replacing "fossero": 88.2%
The average odds is: 4672.6926290573665

The average odds when predicting correctly is: 5297.7949670752505
The average odds when predicting falsely is: 0.31752624571361254

The average probability for fossero was: 0.4588860958504854
The average probability for erano was: 0.04789822937489373

The average probability for fossero when predicting correctly is: 0.5145278735758758
The average probability for erano when predicting correctly is: 0.028574973624000327
The average probability for fossero when predicting falsely is: 0.04298738437765175
The average probability for erano when predicting falsely is: 0.19233137829258848


### erano was replaced ###
Accuracy when replacing "erano": 98.8%
The average odds is: 801271.4901295332

The average odds when predicting correctly is: 811003.5283387073
The average odds when predicting falsely is: 0.3442408644507678

The average probability for erano was: 0.40745257815647773
The 

In [82]:
experiment_1(corpus, 'sono', 'siano', 500)

### sono was replaced ###
Accuracy when replacing "sono": 99.8%
The average odds is: 902.499674573009

The average odds when predicting correctly is: 904.306779757173
The average odds when predicting falsely is: 0.7541876751871859

The average probability for sono was: 0.4724690455650628
The average probability for siano was: 0.008801220344894339

The average probability for sono when predicting correctly is: 0.4728989485126519
The average probability for siano when predicting correctly is: 0.008133446691860825
The average probability for sono when predicting falsely is: 0.25794747471809387
The average probability for siano when predicting falsely is: 0.34202027320861816


### siano was replaced ###
Accuracy when replacing "siano": 27.2%
The average odds is: 6.943140354481208

The average odds when predicting correctly is: 24.96537005337328
The average odds when predicting falsely is: 0.20956002742263588

The average probability for siano was: 0.21949496043055083
The average probabilit

In [83]:
experiment_1(corpus, 'erano', 'fossero', 500)

### erano was replaced ###
Accuracy when replacing "erano": 96.8%
The average odds is: 4789.918643201053

The average odds when predicting correctly is: 4948.241735659056
The average odds when predicting falsely is: 0.6450963464581804

The average probability for erano was: 0.21596862357176289
The average probability for fossero was: 0.006847677124260147

The average probability for erano when predicting correctly is: 0.22195900099920945
The average probability for fossero when predicting correctly is: 0.00524275290184653
The average probability for erano when predicting falsely is: 0.03475970639150461
The average probability for fossero when predicting falsely is: 0.055396634852272086


### fossero was replaced ###
Accuracy when replacing "fossero": 52.4%
The average odds is: 9.671214116278168

The average odds when predicting correctly is: 18.152567811046676
The average odds when predicting falsely is: 0.33459786405400954

The average probability for fossero was: 0.07874987738008259


In [198]:
for item in list(topk_indices):
    print(tokenizer.convert_ids_to_tokens(item.item()), logits[item.item()].item())

Non 0.5619245767593384
non 0.2179812639951706
è 0.017772043123841286
no 0.009306162595748901
No 0.008768565952777863


In [109]:
import random

def replace_compare_2(corpus, n, word_1, word_2, x):

    corp_1 = find_instances(corpus, word_1)
    corp_1 = random.sample(corp_1, n)
    corp_1_n_correct = 0
    
    samples = []

    for sentence in corp_1:
        correct_prob = test_sentence(sentence, word_1, word_1)
        false_prob = test_sentence(sentence, word_1, word_2)
        if correct_prob == None:
            print(sentence)
            continue
        if false_prob == None:
            print(sentence)
            continue


        if x * correct_prob >= false_prob:
            corp_1_n_correct += 1
            
    corp_1_n_false = len(corp_1)-corp_1_n_correct
    if corp_1_n_false == 0:
        corp_1_n_false = 0.0000001
     
    print(f'Accuracy when replacing "{word_1}": {100*corp_1_n_correct/len(corp_1)}%')
    
    if len(samples) > 5:
        return samples[:5]
    return samples
    
    

def experiment_2(corpus, word_1, word_2, n, x):
    samples = []
    samples.append(replace_compare_2(corpus, n, word_1, word_2, 1/x))
    samples.append(replace_compare_2(corpus, n, word_2, word_1, x))
    print(samples)

In [88]:
experiment_2(corpus, 'sono', 'siano', 10, 10)

### sono was replaced ###
Accuracy when replacing "sono": 100.0%
The average odds is: 1157.1675375276545

The average odds when predicting correctly is: 1157.1675375276545
The average odds when predicting falsely is: 0.0

The average probability for sono was: 0.4579587295591773
The average probability for siano was: 0.00396752415610365

The average probability for sono when predicting correctly is: 0.4579587295591773
The average probability for siano when predicting correctly is: 0.00396752415610365
The average probability for sono when predicting falsely is: 0.0
The average probability for siano when predicting falsely is: 0.0


### siano was replaced ###
Accuracy when replacing "siano": 95.0%
The average odds is: 2.3453953988131233

The average odds when predicting correctly is: 2.4685262204107725
The average odds when predicting falsely is: 0.005909788457789384

The average probability for siano was: 0.188029572701865
The average probability for sono was: 0.4312951381831078

The ave

In [89]:
experiment_2(corpus, 'erano', 'fossero', 100)

### erano was replaced ###
Accuracy when replacing "erano": 100.0%
The average odds is: 3657.332465502502

The average odds when predicting correctly is: 3657.332465502502
The average odds when predicting falsely is: 0.0

The average probability for erano was: 0.1887540190281468
The average probability for fossero was: 0.007481184508735399

The average probability for erano when predicting correctly is: 0.1887540190281468
The average probability for fossero when predicting correctly is: 0.007481184508735399
The average probability for erano when predicting falsely is: 0.0
The average probability for fossero when predicting falsely is: 0.0


### fossero was replaced ###
Accuracy when replacing "fossero": 99.0%
The average odds is: 6.815849145987573

The average odds when predicting correctly is: 6.884647669100133
The average odds when predicting falsely is: 0.00479535784401859

The average probability for fossero was: 0.08284206437712918
The average probability for erano was: 0.05442053

In [93]:
experiment_2(corpus, 'sono', 'siano', 10, 500)

### sono was replaced ###
Accuracy when replacing "sono": 100.0%
The average odds is: 444.5960143193952

The average odds when predicting correctly is: 444.5960143193952
The average odds when predicting falsely is: 0.0

The average probability for sono was: 0.44315625164308586
The average probability for siano was: 0.005199900197476381

The average probability for sono when predicting correctly is: 0.44315625164308586
The average probability for siano when predicting correctly is: 0.005199900197476381
The average probability for sono when predicting falsely is: 0.0
The average probability for siano when predicting falsely is: 0.0


### siano was replaced ###
Accuracy when replacing "siano": 100.0%
The average odds is: 11.861151553190883

The average odds when predicting correctly is: 11.861151553190883
The average odds when predicting falsely is: 0.0

The average probability for siano was: 0.1770077091103303
The average probability for sono was: 0.31462434413842855

The average probabi

In [99]:
experiment_2(corpus, 'sono', 'siano', 50, 10)

### sono was replaced ###
Accuracy when replacing "sono": 100.0%
The average odds is: 1070.6849533909729

The average odds when predicting correctly is: 1070.6849533909729
The average odds when predicting falsely is: 0.0

The average probability for sono was: 0.5017051460250513
The average probability for siano was: 0.019917086907566953

The average probability for sono when predicting correctly is: 0.5017051460250513
The average probability for siano when predicting correctly is: 0.019917086907566953
The average probability for sono when predicting falsely is: 0.0
The average probability for siano when predicting falsely is: 0.0


### siano was replaced ###
Accuracy when replacing "siano": 74.0%
The average odds is: 8.467180789604717

The average odds when predicting correctly is: 11.426849577503251
The average odds when predicting falsely is: 0.04350808558580299

The average probability for siano was: 0.27010721675604654
The average probability for sono was: 0.3316563809957975

The a

In [100]:
experiment_2(corpus, 'sono', 'siano', 50, 0.1)

### sono was replaced ###
Accuracy when replacing "sono": 92.0%
The average odds is: 323.50092208056054

The average odds when predicting correctly is: 351.41105461265704
The average odds when predicting falsely is: 2.534397961450886

The average probability for sono was: 0.31675642627999523
The average probability for siano was: 0.014397617363618167

The average probability for sono when predicting correctly is: 0.33506020045024343
The average probability for siano when predicting correctly is: 0.003277118584366156
The average probability for sono when predicting falsely is: 0.10626302332214088
The average probability for siano when predicting falsely is: 0.1422833533250163


### siano was replaced ###
Accuracy when replacing "siano": 10.0%
The average odds is: 5.8085528994795705

The average odds when predicting correctly is: 50.31306123330175
The average odds when predicting falsely is: 0.8636075290548848

The average probability for siano was: 0.21563974740267855
The average probab

In [106]:
experiment_2(corpus, 'sono', 'siano', 50, 1/15)

### sono was replaced ###
Accuracy when replacing "sono": 90.0%
The average odds is: 900.9235939409925

The average odds when predicting correctly is: 999.9841444973056
The average odds when predicting falsely is: 9.378638934175594

The average probability for sono was: 0.37969800011720506
The average probability for siano was: 0.007068398124346799

The average probability for sono when predicting correctly is: 0.3900635900338077
The average probability for siano when predicting correctly is: 0.00382853117750162
The average probability for sono when predicting falsely is: 0.2864076908677816
The average probability for siano when predicting falsely is: 0.03622720064595342


### siano was replaced ###
Accuracy when replacing "siano": 12.0%
The average odds is: 12.069433474466841

The average odds when predicting correctly is: 89.09440286585601
The average odds when predicting falsely is: 1.5660285574592274

The average probability for siano was: 0.27474085523618635
The average probabilit

In [126]:
experiment_2(corpus, 'sono', 'siano', 50, 15)

Accuracy when replacing "sono": 84.0%
Accuracy when replacing "siano": 70.0%
[[], []]


In [134]:
for i in [1, 2, 3, 4, 5]:
    experiment_2(corpus, 'è', 'sia', 50, i)

Accuracy when replacing "è": 98.0%
Accuracy when replacing "sia": 36.0%
[[], []]
Accuracy when replacing "è": 94.0%
Accuracy when replacing "sia": 42.0%
[[], []]
Accuracy when replacing "è": 94.0%
Accuracy when replacing "sia": 58.0%
[[], []]
Accuracy when replacing "è": 88.0%
Accuracy when replacing "sia": 60.0%
[[], []]
Accuracy when replacing "è": 92.0%
Accuracy when replacing "sia": 60.0%
[[], []]


In [136]:
for i in [5,7, 10,12, 15]:
    experiment_2(corpus, 'sono', 'siano', 100, i)

Accuracy when replacing "sono": 100.0%
Accuracy when replacing "siano": 47.0%
[[], []]
Accuracy when replacing "sono": 95.0%
Accuracy when replacing "siano": 52.0%
[[], []]
Accuracy when replacing "sono": 94.0%
Accuracy when replacing "siano": 70.0%
[[], []]
Accuracy when replacing "sono": 88.0%
Accuracy when replacing "siano": 69.0%
[[], []]
Accuracy when replacing "sono": 93.0%
Accuracy when replacing "siano": 75.0%
[[], []]


In [114]:
for i in [1,1.5,2,2.5,3,3.5]:
    experiment_2(corpus, 'erano', 'fossero', 300, i)

Accuracy when replacing "erano": 97.0%
Accuracy when replacing "fossero": 51.666666666666664%
[[], []]
Accuracy when replacing "erano": 93.33333333333333%
Accuracy when replacing "fossero": 63.333333333333336%
[[], []]
Accuracy when replacing "erano": 92.66666666666667%
Accuracy when replacing "fossero": 67.33333333333333%
[[], []]
Accuracy when replacing "erano": 91.0%
Accuracy when replacing "fossero": 72.66666666666667%
[[], []]
Accuracy when replacing "erano": 92.0%
Accuracy when replacing "fossero": 73.0%
[[], []]
Accuracy when replacing "erano": 88.66666666666667%
Accuracy when replacing "fossero": 77.66666666666667%
[[], []]


In [135]:
experiment_2(corpus, 'è', 'sia', 50, 2.5)
experiment_2(corpus, 'sono', 'siano', 50, 2.5)
experiment_2(corpus, 'erano', 'fossero', 50, 2.5)

Accuracy when replacing "è": 96.0%
Accuracy when replacing "sia": 42.0%
[[], []]
Accuracy when replacing "sono": 96.0%
Accuracy when replacing "siano": 48.0%
[[], []]
Accuracy when replacing "erano": 90.0%
Accuracy when replacing "fossero": 68.0%
[[], []]


In [137]:
experiment_2(corpus, 'è', 'sia', 150, 2.5)
experiment_2(corpus, 'sono', 'siano', 150, 2.5)
experiment_2(corpus, 'erano', 'fossero', 150, 2.5)

Accuracy when replacing "è": 96.0%
Accuracy when replacing "sia": 40.666666666666664%
[[], []]
Accuracy when replacing "sono": 97.33333333333333%
Accuracy when replacing "siano": 41.333333333333336%
[[], []]
Accuracy when replacing "erano": 92.0%
Accuracy when replacing "fossero": 68.66666666666667%
[[], []]
